## BPR: Bayesian Personalized Ranking from Implicit Feedback

Ref: 
* https://arxiv.org/pdf/1205.2618
* https://medium.com/radon-dev/implicit-bayesian-personalized-ranking-in-tensorflow-b4dfa733c478

In [ ]:
WIP

In [16]:
import tensorflow as tf
# tf.enable_eager_execution()
# import tensorflow.contrib.eager as tfe
import pandas as pd
import numpy as np
import scipy.sparse as sp
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

In [17]:
#---------------------------
# LOAD AND PREPARE THE DATA
#---------------------------

# Load the dataframe from a tab separated file.
df = pd.read_csv('data/movielens/ml-latest-small/ratings.csv', sep=',')
    
# Add column names
df = df.drop(df.columns[3], axis=1)
df_movie = pd.read_csv('data/movielens/ml-latest-small/movies.csv')

In [18]:
movie_encoder = LabelEncoder()
df_movie['movie_id']=movie_encoder.fit_transform(df_movie.movieId)

In [19]:
df_movie.head()

,movieId,title,genres,movie_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [20]:

# Drop any rows with missing values
df = df.dropna()

# Drop any rows with 0 rating
df = df.loc[df.rating != 0]

# Convert movies into numerical IDs
df['user_id'] = df['userId'].astype("category").cat.codes
df['movie_id'] = movie_encoder.transform(df['movieId'].values)

# Create a lookup frame so we can get the movie
# names back in readable form later.
item_lookup = df[['movie_id', 'movieId']].drop_duplicates()
item_lookup['movie_id'] = item_lookup.movie_id.astype(str)

# We drop our old user and item columns
df = df.drop(['userId', 'movieId'], axis=1)

# Drop any rows with 0 rating
df = df.loc[df.rating != 0]

# Create lists of all users, movies and ratings
users = list(np.sort(df.user_id.unique()))
movies = list(np.sort(df_movie.movie_id.unique()))
ratings = list(df.rating)
print(f"#users: {len(users):,}, #items: {len(movies):,} #ratings: {len(ratings):,}" )

# Get the rows and columns for our new matrix
rows = df.user_id.astype(float)
cols = df.movie_id.astype(float)

# Contruct a sparse matrix for our users and items containing number of ratings
data_sparse = sp.csr_matrix((ratings, (rows, cols)), shape=(len(users), len(movies)))

# Get the values of our matrix as a list of user ids
# and item ids. Note that our litsts have the same length
# as each user id repeats one time for each rated movie.
uids, iids = data_sparse.nonzero()

#users: 610, #items: 9,742 #ratings: 100,836


In [22]:
title_vectorizer = TfidfVectorizer()
movie_titles = title_vectorizer.fit_transform(df_movie.title)
genre_vectorizer = TfidfVectorizer()
movie_genre = genre_vectorizer.fit_transform(df_movie.genres)
print(movie_titles.shape, movie_genre.shape)
#movie_features = sp.hstack((movie_titles,movie_genre))
movie_features = movie_genre
movie_features = movie_features.tocsr()
movie_item_features = movie_features[iids,:]

(9742, 9269) (9742, 24)


In [23]:
movie_item_features

<100836x24 sparse matrix of type '<class 'numpy.float64'>'
	with 292687 stored elements in Compressed Sparse Row format>

In [7]:
movie_item_features[0,:].nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([9277, 9273, 9272, 9271, 9270, 8367, 7851,  116], dtype=int32))

In [26]:
non_zero_rows, non_zero_cols = movie_item_features.nonzero()

In [30]:
non_zero_cols

array([8, 4, 3, ..., 5, 0, 7], dtype=int32)

In [31]:
cols_list=[]
value_list=[]
for r in range(movie_item_features.shape[0]):
    cols_list.append([])
    value_list.append([])    
for (nzr, nzc) in zip(non_zero_rows, non_zero_cols):
    value = movie_item_features[nzr,nzc]
    cols_list[nzr].append(nzc)
    value_list[nzr].append(value)

        

In [38]:
#-------------
# HYPERPARAMS
#-------------

epochs = 50
# How many (u,i,j) triplets we sample for each batch
samples = 150
batches = math.ceil(uids.shape[0]/samples)
# batches = min(batches,30)
num_factors = 64 # Number of latent features

# Independent lambda regularization values 
# for user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

# Our learning rate 
lr = 0.005



## Eager way to iterator through dataset

In [11]:
# import tensorflow as tf
# tf.enable_eager_execution()
# import tensorflow.contrib.eager as tfe
# uids=list(range(10))
# iids=list(range(10,20))
# train_data = tf.data.Dataset.from_tensor_slices((uids, iids))
# train_data = train_data.shuffle(len(uids))
# train_data = train_data.batch(2)

# for eps in range(3):
#     batch_id = 0
#     for x, y in train_data:
#         print(f"eps: {eps}, batch: {batch_id}, data: {x}, {y} ")
#         batch_id+=1 

In [39]:
#-------------------------
# TENSORFLOW GRAPH
#-------------------------

# Set up our Tensorflow graph
graph = tf.Graph()

def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random_uniform([size, dim], -std, std), name=name)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

In [40]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [3]:
import itertools
def gen():
    for i in itertools.count(1):
        yield (i, [1] * i)

ds = tf.data.Dataset.from_generator(
    gen, (tf.int64, tf.int64), (tf.TensorShape([]), tf.TensorShape([None])))

for value in ds.take(2):
    print(value)

(<tf.Tensor: id=97, shape=(), dtype=int64, numpy=1>, <tf.Tensor: id=98, shape=(1,), dtype=int64, numpy=array([1])>)
(<tf.Tensor: id=101, shape=(), dtype=int64, numpy=2>, <tf.Tensor: id=102, shape=(2,), dtype=int64, numpy=array([1, 1])>)


In [10]:
import tensorflow as tf
tf.enable_eager_execution()
a=[[1,2],[3]]
b=[[0.1,0.2],[0.3]]
c=np.array([0,0])
d=np.array([1,3])
def gen():
    for w, x, y, z in zip(a,b,c,d):
        yield (w,x,y,z)

train_data = tf.data.Dataset.from_generator(gen,
                                                (tf.int64, tf.float64,tf.int64,tf.int64), 
                                                (None,None,None,None))

for value in train_data.take(2):
    print(value)


(<tf.Tensor: id=319, shape=(2,), dtype=int64, numpy=array([1, 2])>, <tf.Tensor: id=320, shape=(2,), dtype=float64, numpy=array([0.1, 0.2])>, <tf.Tensor: id=321, shape=(), dtype=int64, numpy=0>, <tf.Tensor: id=322, shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: id=327, shape=(1,), dtype=int64, numpy=array([3])>, <tf.Tensor: id=328, shape=(1,), dtype=float64, numpy=array([0.3])>, <tf.Tensor: id=329, shape=(), dtype=int64, numpy=0>, <tf.Tensor: id=330, shape=(), dtype=int64, numpy=3>)


In [49]:
uids

array([  0,   0,   0, ..., 609, 609, 609], dtype=int32)

In [44]:

with graph.as_default():
    '''
    Loss function: 
    -SUM ln σ(xui - xuj) + λ(w1)**2 + λ(w2)**2 + λ(w3)**2 ...
    ln = the natural log
    σ(xuij) = the sigmoid function of xuij.
    λ = lambda regularization value.
    ||W||**2 = the squared L2 norm of our model parameters.
    
    '''
    train_data = tf.data.Dataset.from_tensor_slices((uids, iids, cols_list, value_list))
    train_data = train_data.batch(samples)
    train_data = train_data.prefetch(4)
    iterator = train_data.make_initializable_iterator()
    movie_features_tf = convert_sparse_matrix_to_sparse_tensor(movie_features)
    u, i, item_col, item_values = iterator.get_next()
    
    j = tf.random_uniform(tf.shape(i), minval=0, maxval=len(movies), dtype=tf.int32)
    i_features = tf.sparse.SparseTensor()
    j_features = tf.sparse.slice(movie_features_tf,j)
    # User feature embedding
    user_factors = init_variable(len(users), num_factors, "user_factors") # V matrix
    u_factors = tf.nn.embedding_lookup(user_factors, u)
    # Known and unknown item embeddings
    item_factors = init_variable(len(movies), num_factors, "item_factors") # V matrix
    item_feature_weights = init_variable(len(movie_features.shape[1]), num_factors, "item_feature_weights") # V matrix
    i_identity_factors = tf.nn.embedding_lookup(item_factors, i)
    j_identity_factors = tf.nn.embedding_lookup(item_factors, j)
    i_features_factors = tf.sparse.sparse_dense_matmul(i_features,item_feature_weights)
    j_features_factors = tf.sparse.sparse_dense_matmul(j_features,item_feature_weights)
    
    i_factors = i_identity_factors + i_features_factors
    j_factors = j_identity_factors + j_features_factors

    # i and j bias embeddings.
    item_bias = init_variable(len(movies), 1, "item_bias")
    i_bias = tf.nn.embedding_lookup(item_bias, i)
    i_bias = tf.squeeze(i_bias)
    j_bias = tf.nn.embedding_lookup(item_bias, j)
    j_bias = tf.squeeze(j_bias)

    # Calculate the dot product + bias for known and unknown
    # item to get xui and xuj.
    ui = tf.reduce_sum(u_factors * i_factors, axis=1)
    xui = i_bias + ui
    uj = tf.reduce_sum(u_factors * j_factors, axis=1)
    xuj = j_bias + uj

    # We calculate xuij.
    xuij = xui - xuj

    # Calculate the mean AUC (area under curve).
    # if xuij is greater than 0, that means that 
    # xui is greater than xuj (and thats what we want).
    u_auc = tf.reduce_mean(tf.cast(xuij > 0,tf.float32))

    # Output the AUC value to tensorboard for monitoring.
    tf.summary.scalar('auc', u_auc)

    # Calculate the squared L2 norm ||W||**2 multiplied by λ.
    l2_norm = tf.add_n([
        lambda_user * tf.reduce_sum(tf.multiply(u_factors, u_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(i_factors, i_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(j_factors, j_factors)),
        lambda_bias * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
        lambda_bias * tf.reduce_sum(tf.multiply(j_bias, j_bias))
        ])

    # Calculate the loss as ||W||**2 - ln σ(Xuij)
    #loss = l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij)))
    loss = -tf.reduce_mean(tf.log(tf.sigmoid(xuij))) + l2_norm
    
    # Train using the Adam optimizer to minimize 
    # our loss function.
    opt = tf.train.AdamOptimizer(learning_rate=lr)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()

ValueError: Argument must be a dense tensor: [[8, 4, 3, 2, 1], [19, 4], [21, 5, 0], [16, 21], [16, 21, 5], [12, 21, 0, 4], [5, 19, 4, 1], [22, 0, 7], [22, 0, 7, 19], [22, 4], [23, 0, 19], [4], [4], [4, 1], [7, 4], [9, 20, 0, 1], [21, 5, 7, 4], [9, 20, 0, 1], [4], [21, 5, 0, 7], [22, 7, 19, 4], [19, 3, 1], [5, 0, 8, 4], [21, 0], [4], [21], [9, 20, 21, 0, 1], [7, 4], [22, 7], [21, 19, 4], [0, 19, 4, 1], [23, 0, 7], [23, 7, 1], [21, 5, 0], [12, 21, 5], [15, 8, 3, 2], [21, 5, 7, 4], [16, 21, 0, 1], [15, 8, 3, 2, 1], [9, 20, 8, 4, 3, 2, 1], [21, 0, 1], [21, 0, 19, 1], [9, 20, 21, 0, 1], [19, 4], [15, 8, 3, 1], [16, 7], [0, 19, 1], [7, 19, 8], [7], [8, 3, 1], [15, 3, 2], [15, 3, 2], [15, 8, 3, 2], [15, 7, 3, 2], [15, 3, 2, 1], [15, 3, 1], [15, 8, 3, 2, 1], [7, 4], [0, 1], [7, 4], [15, 8, 4, 3], [4], [16, 21, 5], [22, 7], [16, 21, 5], [9, 20, 7, 3], [9, 20, 21, 0, 1], [8, 4, 1], [9, 20, 0, 1], [0, 19, 8, 4, 1], [0, 1], [9, 20, 21, 5, 7], [22, 0, 7], [9, 20, 0, 1], [5, 7], [9, 20, 12], [12, 5]

In [ ]:
movie_features_tf

In [ ]:
%%time
#------------------
# GRAPH EXECUTION
#------------------

# Run the session. 
session = tf.Session(config=None, graph=graph)
session.run(init)

# This has noting to do with tensorflow but gives
# us a nice progress bar for the training.
progress = tqdm(total=batches*epochs)
idx=np.arange(uids.shape[0])
for _ in range(epochs):
    session.run(iterator.initializer)

    # We run the session.
    _, l, auc = session.run([step, loss, u_auc])
    progress.update(batches)
    progress.set_description('Loss: %.3f | AUC: %.3f' % (l, auc))

progress.close()